<a href="https://colab.research.google.com/github/MohanZhu0623/Sentiment_Analysis/blob/main/TL(SieBERT%26RoBERTa)_ThreeClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install Hugging Face's transformers and datasets libraries
!pip install transformers
!pip install datasets

In [ ]:
# check GPU status
!nvidia-smi

Wed Jul 24 13:09:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0              32W /  70W |   6193MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# check version of transformers
import transformers
print(transformers.__version__)

4.42.4


In [ ]:
from transformers import pipeline  # load pipeline() function from transformers library
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")  # load pretrained SieBERT model ("Sentiment in English")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
!pip uninstall pyarrow -y
!pip install pyarrow==12.0.0
!pip install transformers datasets scikit-learn
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0
  Using cached pyarrow-12.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 12.0.0 which is incompatible.
datasets 2.20.0 requires pyarrow>=15.0.0, but you have pyarrow 12.0.0 which is incompatible.


  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (39.9 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 12.0.0
    Uninstalling pyarrow-12.0.0:
      Successfully uninstalled pyarrow-12.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
datathree = pd.read_excel('/content/python_labelled_data.xlsx')
dataset = Dataset.from_pandas(datathree)

In [ ]:
# spit training_dataset&test_dataset
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
# print first row from training data split
print(train_dataset[0])

{'text': 'Blue Flame is a Mystical SyFi drama, that lets practical FX makeup explore new heights, with  the unlimited characters to be used.', 'class': 1}


In [ ]:
# count number of labels
NUM_LABELS = len(set(train_test_split['test']['class']))
print(set(train_test_split['test']['class']))
print(NUM_LABELS)

{0, 1, -1}
3


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
checkpoint = "siebert/sentiment-roberta-large-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=NUM_LABELS, ignore_mismatched_sizes=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at siebert/sentiment-roberta-large-english and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([3, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
tokenized_train_dataset = tokenized_train_dataset.rename_column("class", "label")
tokenized_test_dataset = tokenized_test_dataset.rename_column("class", "label")

Map:   0%|          | 0/1815 [00:00<?, ? examples/s]

Map:   0%|          | 0/454 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def update_labels(example):
    example['label'] = example['label'] + 1
    return example

tokenized_train_dataset = tokenized_train_dataset.map(update_labels)
tokenized_test_dataset = tokenized_test_dataset.map(update_labels)

Map:   0%|          | 0/1815 [00:00<?, ? examples/s]

Map:   0%|          | 0/454 [00:00<?, ? examples/s]

In [ ]:
from datasets import ClassLabel
new_features = tokenized_test_dataset.features.copy()
new_features["label"] = ClassLabel(names=['neg', 'neu', 'pos'])
tokenized_test_dataset = tokenized_test_dataset.cast(new_features)

Casting the dataset:   0%|          | 0/454 [00:00<?, ? examples/s]

In [ ]:
# define evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# initialize pretrained model with updated classification head
model2 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=NUM_LABELS, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at siebert/sentiment-roberta-large-english and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([3, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# set number of epochs
NUM_EPOCHS = 1

In [ ]:
# train_SieBERT
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.761624,0.623348,0.614234,0.646896,0.659947


TrainOutput(global_step=114, training_loss=0.8735664769222862, metrics={'train_runtime': 117.9291, 'train_samples_per_second': 15.391, 'train_steps_per_second': 0.967, 'total_flos': 119678957240094.0, 'train_loss': 0.8735664769222862, 'epoch': 1.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
trainer.save_model('/content/drive/My Drive/trained_siebert_model')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# specify path of pretrained model
checkpoint = "roberta-large"  # RoBERTa-large
# load pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# initialize pretrained model with updated classification head
model3 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=NUM_LABELS, ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# train RoBERTa
trainer = Trainer(
    model=model3,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.709376,0.647577,0.650377,0.652251,0.662139


TrainOutput(global_step=114, training_loss=0.9228880363598204, metrics={'train_runtime': 157.1105, 'train_samples_per_second': 11.552, 'train_steps_per_second': 0.726, 'total_flos': 119678957240094.0, 'train_loss': 0.9228880363598204, 'epoch': 1.0})